# Use the geostatistical toolbox container

This short tutorial shows, how you can interact with the container using the help of `toolbox-runner`. 
The runner is still under development, and interacting with the result files is still a bit messy.
Right now, the results are placed into a temporary folder and the result files are extracted by hand. In  stable release,
`toolbox-runner` will have helper functions for that purpose.

In [1]:
from toolbox_runner import list_tools
import io
import tempfile
import json
import pandas as pd
import xarray as xr
import numpy as np
from pprint import pprint

Use `list_tools` to get a list or dict of all tool images present on your system. Additionally, a CSV file with example data is loaded.
The `list_tools` function accepts also a prefix, if you did not prefix your tool container with `'tbr_'`.

In [2]:
tools = list_tools(as_dict=True)
pprint(tools)

{'cross-validation': cross-validation: Leave-one-out cross-validation  FROM tbr_skgstat:latest VERSION: 1.0,
 'kriging': kriging: Kriging interpolation  FROM tbr_skgstat:latest VERSION: 1.0,
 'profile': profile: Dataset Profile  FROM tbr_profile:latest VERSION: 0.2,
 'sample': sample: Sample field data  FROM tbr_skgstat:latest VERSION: 1.0,
 'simulation': simulation: Geostatistical simulation  FROM tbr_skgstat:latest VERSION: 1.0,
 'variogram': variogram: Variogram fitting  FROM tbr_skgstat:latest VERSION: 1.1}


In [3]:
# load some data samples
df = pd.read_csv('in/meuse.csv')
coords = df[['x', 'y']].values
vals = df.lead.values
print(df.head())

ds = xr.open_dataset('in/cmip_prec.nc')
ds

        x       y  lead
0  181072  333611   299
1  181025  333558   277
2  181165  333537   199
3  181298  333484   116
4  181307  333330   117


<xarray.Dataset>
Dimensions:  (x: 50, y: 105)
Dimensions without coordinates: x, y
Data variables:
    prec     (x, y) float32 ...

## General info about tools

Although using `toolbox-runner` is not mandatory and the tool container are self-contained, there are some helpful functions.
You can get structured metadata about each tool:

In [4]:
vario = tools.get('variogram')

print(vario.title)
print(vario.description)
print('\nPARAMETERS\n---------\n')
for key, conf in vario.parameters.items():
    print(f"## {key}")
    print(f"input type:    {conf['type']}")
    print(f"description:   {conf.get('description', 'no description provided')}")
    print()

Variogram fitting
Estimate an empirical variogram and fit a model

PARAMETERS
---------

## coordinates
input type:    file
description:   Pass either a (N, D) shaped numpy array, or a .mat file containing the matrix of observation location coordinates

## values
input type:    file
description:   Pass either a (N, 1) shaped numpy array or a .mat file containing the matrix of observations

## n_lags
input type:    integer
description:   no description provided

## bin_func
input type:    enum
description:   no description provided

## model
input type:    enum
description:   no description provided

## estimator
input type:    enum
description:   no description provided

## maxlag
input type:    string
description:   Can be 'median', 'mean', a number < 1 for a ratio of maximum separating distance or a number > 1 for an absolute distance

## fit_method
input type:    enum
description:   no description provided

## use_nugget
input type:    bool
description:   Enable the nugget parameter

## Variogram

Run the variogram tool to estimate a base variogram, which will then be used for the other tools.
The variogram parameterization is stored in a `'./out/variogram.json'` in the result tarball. Right now, we have to read 
this ourselfs and parse the json

The container does also produce HTML and JSON plotly figures, a PDF figure and a copy of all settings of the `skg.Variogram` instance.

In [5]:
# get the tool
vario = tools.get('variogram')

# run
step = vario.run(result_path='./test', coordinates=coords, values=vals, maxlag='median', model='exponential', bin_func='scott')

# the parameters are stored in variogram.json
vario_params = step.get('variogram.json')
pprint(vario_params)

{'bin_func': 'scott',
 'dist_func': 'euclidean',
 'estimator': 'matheron',
 'fit_method': 'trf',
 'fit_sigma': None,
 'maxlag': 1372.6660191029719,
 'model': 'exponential',
 'n_lags': 10,
 'normalize': False,
 'use_nugget': False,
 'verbose': False}


## Simulation

Run a geostatistical simulation using the variogram from the last step. The simulation result is returned as `.mat` files. 
Right now, we need to put these into a file-like object and read them using numpy. That will be improved in the future.

The current version of the tool can only return the mean and standard deviation of the simulated fields. A future version
will have the optional export of a netCDF containing every simulation.

In [6]:
simu = tools.get('simulation')

# run a simulation
step = simu.run(result_path='./test', coordinates=coords, values=vals, variogram=vario_params, n_simulations=10, grid='50x50')

# print the log
print(step.log)

# print all outputs
print(step.outputs)

# .dat files can be loaded to numpy. Load the mean of all simulations
mean = step.get('simulation_mean.dat')

mean

Estimating variogram...
exponential Variogram
---------------------
Estimator:         matheron
Effective Range:   1372.67
Sill:              17740.72
Nugget:            0.00
        
Starting 10 iterations seeded 42

['STDERR.log', 'STDOUT.log', 'result.json', 'simulation_mean.dat', 'simulation_std.dat', 'variogram.html', 'variogram.json', 'variogram.pdf']


array([[171.82328018, 177.51347383, 164.87992787, ..., 121.29299786,
        146.47249759, 181.21038282],
       [173.66173153, 187.09362293, 169.08039522, ..., 129.47851113,
        138.09837756, 168.64932544],
       [192.41124558, 199.84261111, 184.90009063, ..., 178.66673437,
        147.14080855, 184.20576035],
       ...,
       [154.72179719, 184.44261002, 185.62482896, ...,  98.83145353,
        112.23031823, 130.52692855],
       [192.51765686, 180.46835029, 158.42274169, ...,  92.45844056,
        106.70579276, 125.39187714],
       [165.30685215, 169.39535479, 187.13380149, ..., 112.48571592,
        116.8817058 , 129.10454359]])

#### Plot the simulation result

The tool does not yet include an automatic plot as result (like variogram), but we can easily build this using plotly.

In [7]:
import plotly.offline as py
import plotly.graph_objects as go
py.init_notebook_mode(connected=True)

In [8]:
fig = go.Figure()

xx = np.mgrid[coords[:,0].min():coords[:,0].max():50j]
yy = np.mgrid[coords[:,1].min():coords[:,1].max():50j]

fig.add_trace(go.Heatmap(z=mean.T, x=xx, y=yy))
fig.add_trace(go.Scatter(x=coords[:,0], y=coords[:,1], mode='markers', marker=dict(color=vals)))
py.iplot(fig)

## Kriging

We can pass the same variogram to the kriging tool and compare interpolation and simulation

In [9]:
krig = tools.get('kriging')

# run kringing with the same variogram
step = krig.run(result_path='./test', coordinates=coords, values=vals, variogram=vario_params, algorithm='ordinary', grid='50x50')

# print outputs
print(f'OUTPUTS: {step.outputs}')

# get the kriging field
krig_field = step.get('kriging.dat')

OUTPUTS: ['STDERR.log', 'STDOUT.log', 'kriging.dat', 'result.json', 'sigma.dat', 'variogram.html', 'variogram.json', 'variogram.pdf']


In [10]:
fig = go.Figure()

xx = np.mgrid[coords[:,0].min():coords[:,0].max():50j]
yy = np.mgrid[coords[:,1].min():coords[:,1].max():50j]

fig.add_trace(go.Heatmap(z=krig_field.T, x=xx, y=yy))
fig.add_trace(go.Scatter(x=coords[:,0], y=coords[:,1], mode='markers', marker=dict(color=vals)))
py.iplot(fig)

## Sampling

We can use the geostatistics toolbox for sample, as well. We need any kind of field input and specifiy the number of samples. Right now, the toolbox supports random sampling and sampling on a regular grid.

In [11]:
sample = tools.get('sample')

# run the step, pass the netCDF DataArray values in. Passing netCDF directly is currently not yet supported
step = sample.run(result_path='./test', field=ds.prec.values, sample_size=200, method='random', seed=42)

sam_coords = step.get('coordinates.dat')
sam_values = step.get('values.dat')

print(sam_coords[:5], sam_values[:5])

[[46. 37.]
 [30. 52.]
 [36. 46.]
 [20.  9.]
 [19.  9.]] [1.56643319 1.21410108 1.09151483 0.90275711 0.79714686]


These coordinates and values can now be used in the variogram tool again. 

## Cross-validation

The geostatistics toolbox can also be used for cross-validation of variogram. Here, we will use the new samples from the last part and derive two new variograms. Both are then assessed using cross-validation

In [12]:
cv = tools.get('cross-validation')

# calculate the two variograms
v1 = vario.run(result_path='./test', coordinates=sam_coords, values=sam_values, maxlag='median', n_lags=25, model='exponential')
v2 = vario.run(result_path='./test', coordinates=sam_coords, values=sam_values, maxlag='median', n_lags=25, model='gaussian')

v1_params = v1.get('variogram.json')
v2_params = v2.get('variogram.json')

pprint(v1_params)
pprint(v2_params)

{'bin_func': 'even',
 'dist_func': 'euclidean',
 'estimator': 'matheron',
 'fit_method': 'trf',
 'fit_sigma': None,
 'maxlag': 36.61966684720111,
 'model': 'exponential',
 'n_lags': 25,
 'normalize': False,
 'use_nugget': False,
 'verbose': False}
{'bin_func': 'even',
 'dist_func': 'euclidean',
 'estimator': 'matheron',
 'fit_method': 'trf',
 'fit_sigma': None,
 'maxlag': 36.61966684720111,
 'model': 'gaussian',
 'n_lags': 25,
 'normalize': False,
 'use_nugget': False,
 'verbose': False}


In [13]:
cv1 = cv.run(result_path='./test', coordinates=sam_coords, values=sam_values, variogram=v1_params, measure='rmse')
cv2 = cv.run(result_path='./test', coordinates=sam_coords, values=sam_values, variogram=v2_params, measure='rmse')

print(cv1.log)
print(cv2.log)

Estimating variogram...
exponential Variogram
---------------------
Estimator:         matheron
Effective Range:   36.62
Sill:              0.61
Nugget:            0.00
        
Took 2.65 seconds.
RMSE: 1.0448182745681351

Estimating variogram...
gaussian Variogram
------------------
Estimator:         matheron
Effective Range:   36.62
Sill:              0.71
Nugget:            0.00
        
Took 3.04 seconds.
RMSE: 17.731107405500516



You can see, that the cross-validation of the variogram models clearly favors the exponential over the gaussian model for the random sample laken in the last step.